In [2]:
import yaml

In [40]:


class ParseError(BaseException): pass

class Rule:
    PARSER = {
        'allowed_args': {'name', 'subject', 'must', 'must-not'}
    }
    
    def __init__(self, name, subject, must):
        self.name = name
        self.subject = subject
        self.must = must
    
    @classmethod
    def parse(cls, opts):
        unknown_args = set(opts.keys()) - __class__.PARSER['allowed_args']
        if len(unknown_args) > 0:
            raise ParseError(f'Unexpected rule arguments "{unknown_args}" for rule "{rule_name}"')
        
        if 'name' not in opts:
            raise ParseError(f'No name defined for rule: {opts}')
        name = opts['name']
            
        if 'subject' not in opts:
            raise ParseError(f'No subject defined for rule: {name}')
        subject = RuleSubject.parse(opts['subject'], rule_name=name)
        
        if not ('must' in opts or 'must-not' in opts):
            raise ParseError(f'Rule "{name}" does not specify a "must" or "must-not" condition')
        must = 'must parser'
        
        return Rule(name, subject, must)
        
class RuleSubject:
    PARSER = {
        'allowed_args': {'type', 'tags', 'relates-to'},
        'node_types': {'node', 'snapshot', 'source'}
    }
    
    def __init__(self, node_type = 'node', tags=None, relates_to=None):
        self.node_type = node_type
        self.tags = tags
        self.relates_to = relates_to
        
    @classmethod
    def parse(cls, opts, rule_name=''):
        unknown_args = set(opts.keys()) - __class__.PARSER['allowed_args']
        if len(unknown_args) > 0:
            raise ParseError(f'Unexpected subject arguments "{unknown_args}" for rule "{rule_name}"')
        
        if not isinstance(opts, dict):
            raise ParseError(f'Expecting subject arguments for rule "{rule_name}"')
        
        node_type = opts.get('type', 'node')
        if node_type not in __class__.PARSER["node_types"]:
            raise ParseError(f'Unknown subject type "{node_type}" for rule "{rule_name}".  Expecting one of {__class__.PARSER["node_types"]}')

        return RuleSubject(node_type=node_type, tags=opts.get('tags'), relates_to=opts.get('relates-to'))
        
class RuleMust:
    PARSER = {
        'allowed_args': {'match-name', 'have-tag', 'relationship', 'relationship-required', 'related-tags'}
    }
    
    def __init__(self):
        pass
    
    @classmethod
    def parse(cls, opts, rule_name=''):
        unknown_args = set(opts.keys()) - __class__.PARSER['allowed_args']
        if len(unknown_args) > 0:
            raise ParseError(f'Unexpected must arguments "{unknown_args}" for rule "{rule_name}"')
        
        if not isinstance(opts, dict):
            raise ParseError(f'Expecting must arguments for rule "{rule_name}"')
            
        
# Rule.parse(name='a name', subject='a subject', amust='a must')

Rule.parse({
    'name': 'a name',
    'subject': {
       'type': 'node',
    },
    # 'must': 'a must',
    'must-not': 'a must not'
})
    


SyntaxError: invalid syntax (3267601020.py, line 61)

In [46]:
import json
with open('manifest.json') as f:
    manifest = json.loads(f.read())
manifest

{'metadata': {'dbt_schema_version': 'https://schemas.getdbt.com/dbt/manifest/v3.json',
  'dbt_version': '0.21.0',
  'generated_at': '2021-10-20T00:45:30.948361Z',
  'invocation_id': 'a17ba2f0-1144-4b7c-a62c-962dc1fa0c10',
  'env': {},
  'project_id': '4ce5991bf627be9631255c1a7a048b06',
  'user_id': None,
  'send_anonymous_usage_stats': False,
  'adapter_type': 'snowflake'},
 'nodes': {'model.itk_dw.dim_cv_engagements': {'raw_sql': "WITH dim_engagements AS (\n  SELECT * FROM {{ ref('dim_engagements') }}\n),\n\ncurrent_values AS (\n  SELECT *\n  FROM\n    dim_engagements\n  WHERE\n    is_current\n),\n\nfinal AS (\n  SELECT\n    dim_engagements.engagement_sk,\n    {{ dbt_utils.star(\n         ref('dim_engagements'),\n         relation_alias = 'current_values',\n         except=[\n           'ENGAGEMENT_SK',\n           'VALID_FROM',\n           'VALID_TO',\n           'IS_CURRENT'\n         ]\n       )\n    }}\n  FROM\n    dim_engagements\n  INNER JOIN\n    current_values\n    ON\n      d

In [47]:
manifest.keys()

dict_keys(['metadata', 'nodes', 'sources', 'macros', 'docs', 'exposures', 'selectors', 'disabled', 'parent_map', 'child_map'])

In [57]:
manifest['nodes'].keys()
#manifest['nodes']['model.itk_dw.dim_cv_engagements']
manifest['nodes']['snapshot.itk_dw.snap_wecoach__journey']

{'raw_sql': "\n   {{\n        config(\n          target_database=env_var('DBT_DB_SNAPSHOTS', 'UNDEFINED__DBT_DB_SNAPSHOTS'),\n          target_schema='wecoach',\n          alias='journey',\n          unique_key='id',\n          strategy='timestamp',\n          updated_at='_fivetran_synced_ntz',\n        )\n    }}\n\n    SELECT\n      *,\n      CONVERT_TIMEZONE('UTC', _fivetran_synced)::TIMESTAMP AS _fivetran_synced_ntz,\n      CURRENT_TIMESTAMP()::TIMESTAMP AS dbt_snapshot_at\n    FROM\n      {{ source('ft_wecoach', 'journey_c') }}\n\n",
 'compiled': True,
 'resource_type': 'snapshot',
 'depends_on': {'macros': [], 'nodes': ['source.itk_dw.ft_wecoach.journey_c']},
 'config': {'enabled': True,
  'alias': 'journey',
  'schema': 'snapshots',
  'database': None,
  'tags': ['snapshot'],
  'meta': {},
  'materialized': 'snapshot',
  'persist_docs': {},
  'quoting': {},
  'column_types': {},
  'full_refresh': None,
  'on_schema_change': 'ignore',
  'strategy': 'timestamp',
  'unique_key': 'id

In [53]:
manifest['sources'].keys()
manifest['sources']['source.itk_dw.ft_wecoach.account']

{'fqn': ['itk_dw', 'wecoach', 'ft_wecoach', 'account'],
 'database': 'ITK_TEST_STERLING_RAW',
 'schema': 'ft_sf_wecoach',
 'unique_id': 'source.itk_dw.ft_wecoach.account',
 'package_name': 'itk_dw',
 'root_path': '/dbt-runner/dbt',
 'path': 'snapshots/wecoach/sources.yml',
 'original_file_path': 'snapshots/wecoach/sources.yml',
 'name': 'account',
 'source_name': 'ft_wecoach',
 'source_description': 'weCoach Salesforce data, synced via Fivetran.  Use in dbt snapshots.\n',
 'loader': '',
 'identifier': 'account',
 'resource_type': 'source',
 'quoting': {'database': None,
  'schema': None,
  'identifier': None,
  'column': None},
 'loaded_at_field': None,
 'freshness': {'warn_after': None, 'error_after': None, 'filter': None},
 'external': None,
 'description': '',
 'columns': {},
 'meta': {},
 'source_meta': {},
 'tags': [],
 'config': {'enabled': True},
 'patch_path': None,
 'unrendered_config': {},
 'relation_name': 'ITK_TEST_STERLING_RAW.ft_sf_wecoach.account',
 'created_at': 1634690

In [61]:
manifest['child_map'].keys()
#manifest['child_map']['model.itk_dw.base_wecoach__account']
manifest['child_map']['source.itk_dw.ft_wecoach.contact']

['snapshot.itk_dw.snap_wecoach__contact']

In [67]:
x = {'include': True, 'exclude': False}
{1, 2, {'a': 1}}

TypeError: unhashable type: 'dict'